# Boundary and Initial data

First, we initialize the new worksheet;
Note: 
1. This tutorial can be found in the source code repository as as `BoundaryAndInitialData.ipynb`. 
   One can directly load this into Jupyter to interactively work with the following code examples.
2. **In the following line, the reference to `BoSSSpad.dll` is required**. 
   You must either set `#r "BoSSSpad.dll"` to something which is appropirate for your computer
   (e.g. `C:\Program Files (x86)\FDY\BoSSS\bin\Release\net5.0\BoSSSpad.dll` if you installed the binary distribution),
   or, if you are working with the source code, you must compile `BoSSSpad` and put it side-by-side to this worksheet file
   (from the original location in the repository, you can use the scripts `getbossspad.sh`, resp. `getbossspad.bat`).

In [ ]:
#r "BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();


 This tutorial demostrates the **definition**, resp. the **import** of 
 data for **boundary and initial values**. 
 
 In order to demonstrate the usage, 
 we employ the exemplaric **Poisson solver**.

In [ ]:
using BoSSS.Application.SipPoisson;

We use the following helper function to create a **template for the multiple solver runs**.

In [ ]:
Func<SipControl> PreDefinedControl = delegate() {
    SipControl c = new SipControl();
 
    c.SetDGdegree(2);
 
    c.GridFunc = delegate() {
        // define a grid of 10x10 cells
        double[] nodes = GenericBlas.Linspace(-1, 1, 11);
        var grd = Grid2D.Cartesian2DGrid(nodes, nodes);
 
        // set the entire boundary to Dirichlet b.c.
        grd.DefineEdgeTags(delegate (double[] X) {
            return BoundaryType.Dirichlet.ToString();
        });
 
        return grd;
    };
 
    c.SetDatabase(wmg.DefaultDatabase);
    c.savetodb = true; 
 
    return c;    
};

Again, we are using the **workflow management**

In [ ]:
BoSSSshell.WorkflowMgm.Init("Demo_BoundaryAndInitialData");

## Textual and Embedded formulas


In [ ]:
SipControl c1 = PreDefinedControl();

Provide **initial data** as a text:

In [ ]:
c1.AddInitialValue("RHS","X => Math.Sin(X[0])*Math.Cos(X[1])",
                        TimeDependent:false);

Finally, all initial data is stored in the 
`AppControl.InitialValues` dictionary and 
all boundary data is stored in the 
`AppControl.BoundaryValues` dictionary.
 
The common interface for all varinats to specify boundary
and initial data is `IBoundaryAndInitialData`

The snippet above is only a shortcut to add a `Formula` object,
which implements the `IBoundaryAndInitialData` interface.

In [ ]:
c1.InitialValues

In [ ]:
c1.InitialValues["RHS"]

In **BoSSSpad**, such objects can also be extracted from 
static methods of classes; note that these should not depend on any other
object in the worksheet.

In [ ]:
Formula BndyFormula = new Formula(
    "BndyValue.BndyFunction",
    false,
"static class BndyValue {"+
"    public static double BndyFunction(double[] X) {"+
"        return 1.0;"+
"    }"+
"}");

In [ ]:
c1.AddBoundaryValue(BoundaryType.Dirichlet.ToString(),
                    "T",
                    BndyFormula);

Creates a Job named **J1** and runs it

In [ ]:
var J1 = c1.RunBatch();

The next line prints the Status of the Job `J1`.

In [ ]:
BoSSSshell.WorkflowMgm.BlockUntilAllJobsTerminate(3600*4);

We can print the Status of the Job `J1`.

In [ ]:
J1.Status

We can also check via a method if the Job `J1` is truly finished

In [ ]:
NUnit.Framework.Assert.IsTrue(J1.Status == JobStatus.FinishedSuccessful);

## 1D Splines

**Splines** can be used to interpolate nodal data onto a DG field;
currently, only 1D is supported.

In [ ]:
SipControl c2 = PreDefinedControl();

In [ ]:
// create test data for the spline
double[] xNodes = GenericBlas.Linspace(-2,2,13);
double[] yNodes = xNodes.Select(x => x*0.4).ToArray();

In [ ]:
var rhsSpline = new Spline1D(xNodes, yNodes,
                             0,
                             Spline1D.OutOfBoundsBehave.Extrapolate);

In [ ]:
/// BoSSScmdSilent
double err = 0;
// test the spline: a line must be interpolated exactly.
foreach(double xtst in GenericBlas.Linspace(-3,3,77)) { 
   double sVal = rhsSpline.Evaluate(new double[] {xtst , 0, 0 }, 0.0);
   double rVal = xtst*0.4;
   err += Math.Abs(sVal - rVal);
}
NUnit.Framework.Assert.Less(err, 1.0e-10, "Slpine implementation fail.");

In [ ]:
c2.AddInitialValue("RHS", rhsSpline);

In [ ]:
var J2 = c2.RunBatch();

In [ ]:
BoSSSshell.WorkflowMgm.BlockUntilAllJobsTerminate(3600*4);

In [ ]:
J2.Status

In [ ]:
/// BoSSScmdSilent
NUnit.Framework.Assert.IsTrue(J2.Status == JobStatus.FinishedSuccessful);


## Interpolating values from other Calculations

For demonstrational purposes, we use the result (i.e. the last time-step) 
of a previous calculation as a right-hand-side for the next calculation.

In [ ]:
var j2Sess = J2.LatestSession;

In [ ]:
j2Sess

In [ ]:
j2Sess.Timesteps

In [ ]:
var lastTimeStep = j2Sess.Timesteps.Last();

We encapsulate the value `T` in the `ForeignGridValue` object,
which allows interpolation between different meshes:

In [ ]:
var newForeignMesh = new ForeignGridValue(lastTimeStep,"T");

In [ ]:
/// Use different mesh in the control file:
SipControl c3 = PreDefinedControl();

In [ ]:
c3.GridFunc = delegate() {
   // define a grid of *triangle* cells
   double[] nodes = GenericBlas.Linspace(-1, 1, 11);
   var grd = Grid2D.UnstructuredTriangleGrid(nodes, nodes);
 
   // set the entire boundary to Dirichlet b.c.
   grd.DefineEdgeTags(delegate (double[] X) {
       return BoundaryType.Dirichlet.ToString();
   });
 
   return grd;
};
// we also save the RHS in the database
c3.AddFieldOption("RHS", SaveOpt: FieldOpts.SaveToDBOpt.TRUE);

In [ ]:
/// finally, we define the RHS:
c3.AddInitialValue("RHS", newForeignMesh);

In [ ]:
/// BoSSScmdSilent
double orgProbe = newForeignMesh.Evaluate(new double[] {0.5,0.5}, 0.0);
double newProbe = lastTimeStep.GetField("T").ProbeAt(new double[] {0.5,0.5});
NUnit.Framework.Assert.Less(Math.Abs(orgProbe - newProbe), 1.0e-10, "Check (1) on ForeignGridValue failed");

In [ ]:
var J3 = c3.RunBatch();

In [ ]:
BoSSSshell.WorkflowMgm.BlockUntilAllJobsTerminate(3600*4);

In [ ]:
J3.Status

In [ ]:
/// BoSSScmdSilent
NUnit.Framework.Assert.IsTrue(J3.Status == JobStatus.FinishedSuccessful);

Since the quadrilateral mesh used for the original
right-hand-side is geometrically embedded in the triangular mesh 
the **interpolation error should be zero** (up to machine precision).

In [ ]:
var firstTimeStep = J3.LatestSession.Timesteps.First();

In [ ]:
DGField RhsOnTriangles = firstTimeStep.GetField("rhs"); // case-insensitive!
DGField RhsOriginal    = lastTimeStep.GetField("T");

In [ ]:
// note: we have to cast DGField to ConventionalDGField in order to use
// the 'L2Distance' function:
((ConventionalDGField)RhsOnTriangles).L2Distance((ConventionalDGField)RhsOriginal)

In [ ]:
/// BoSSScmdSilent
var H1err = ((ConventionalDGField)RhsOnTriangles).H1Distance((ConventionalDGField)RhsOriginal);
NUnit.Framework.Assert.Less(H1err, 1.0e-10, "Check (2) on ForeignGridValue failed.");

## Restart from Dummy-Sessions

Dummy sessions are kind of fake siolver runs, with the only purpose 
of using them for a restart.

In [ ]:
DGField RHSforRestart = firstTimeStep.GetField("RHS");

We save the DG field `RHSforRestart` in the database;
This automatically creates a timestep and a session which host the DG field:

In [ ]:
var RestartTimestep = wmg.DefaultDatabase.SaveTimestep(RHSforRestart);

In [ ]:
RestartTimestep

In [ ]:
RestartTimestep.Session

This time step can be used as a restart value.:

In [ ]:
var c4 = PreDefinedControl();

In [ ]:
c4.InitialValues.Clear();
c4.SetRestart(RestartTimestep);

In [ ]:
var J4 = c4.RunBatch();

In [ ]:
BoSSSshell.WorkflowMgm.BlockUntilAllJobsTerminate(3600*4);

In [ ]:
J4.Status

In [ ]:
/// BoSSScmdSilent
NUnit.Framework.Assert.IsTrue(J4.Status == JobStatus.FinishedSuccessful);

### Note

Since no mesh interpolation is performed for the restart, it is much faster
than `ForeignGridValue`  but less flexible 
(a restart is always performed on the same mesh).

To avoid multiple mesh interpolations (e.g. when multiple runs are required)
one coudl therefore speed up the process by doing the 
mesh interpolation once **use `ProjectFromForeignGrid`** in BoSSSpad and
save the interpolation in a dummy session.